# Diomira NB

This notebook describes the city of DIOMIRA, which simulates NEXT energy and tracking plane response (sensors and electronics).

author: J.J. Gomez-Cadenas


In [ ]:
import datetime

In [ ]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

## DIOMIRA (Calvino, invisible cities)

Leaving there and proceeding for three days toward the east, you reach Diomira,
a city with sixty silver domes, bronze statues of all the gods, streets paved
with lead, a crystal theater, a golden cock that crows every morning on a
tower. All these beauties will already be familiar to the visitor, who has seen
them also in other cities. But the special quality of this city for the man who
arrives there on a September evening, when the days are growing shorter and the
multicolored lamps are lighted all at once at the doors of the food stalls and
from a terrace a woman's voice cries ooh!, is that he feels envy toward those
who now believe they have once before lived an evening identical to this and
who think they were happy, that time.

## The IC city of DIOMIRA

Diomira simulates the response of the energy plane and tracking plane sensors.

The response of the NEXT detector to ionizing radiation (e.g, an interaction produced by a bb decay or a background event such as a photon or alpha particle interaction) is, in general, an ionization trail. For example, a 2.5 MeV electron produced by a bb decay or a photoelectric interaction induced by a high-energy gamma produced in the Bi-214 chain, propagates by some 10-20 cm in dense gas (at a pressure of ~10 bar). Ionization electrons ejected through the electron path will drift towards the anode and will be amplified in the EL grid. Each ionization electron produces up to 1,000 photons (depending on EL field), which are emitted isotropically, thus reaching the sensors in the anode (SiPMs) and in the cathode (PMTs).

The input to DIMORA are the so-called Monte Carlo Raw Data (MCRD) files, produced by the NEXUS simulation. The MCRD files describe the "true response" of the NEXT sensors to the photon irradiation associated with an event. Photons are propagated through the chamber, and those reaching the sensors are eventually converted in a physical signal (photoelectrons or PES). The MCRD files contain vectors of PES for each sensor in the anode (tracking plane) and cathode (energy plane). The sampling rate in the PMTs of the energy plane is 1 ns (to allow a detailed simulation of the response of the front-end electronics, which runs at 40 MHz), while the sampling rate in the SiPMs is 1 mus, corresponding to the sampling rate of 1MHZ.

The response of the energy plane is simulated by the function:

**simulate_pmt_response(event, pmtrd)** which takes as arguments the event number and a vector of raw-data waveforms (input current in photoelectrons, sampled each ns, for each PMT in the energy plane). 

while the response of the tracking plane is simulated by the function:

**simulate_sipm_response(event, sipmrd, sipms_noise_sampler)** which takes as arguments the event number a vector of raw-data waveforms (input current in photoelectrons, sampled each mus, for each SiPM in the energy plane), and an instance of a noiser class that adds noise to each SiPM. 

Both functions are described below in more detail below.

The output of **simulate_pmt_response** is a vector of containing NPMT waveforms (where NPMT is the number of PMTs in the energy plane). Each waveform is a vector of (tipically) 48,000 samples, corresponding to a DAQ window of 1.2 ms at a sampling rate of 40 MHz. The RWFs represent the response of the energy plane front-end electronics (FEE) to the input photoelectron current. The RWF include the effect of the FEE (convolution with LPF and HPF filters), the effect of the DAQ (sampling at 25 ns), and add the noise of FEE and DAQ. The details of the simulation are described elswhere. 

The RWFs are stored in a 3D array called 
**pmtrwf([event] [npmt] [waveform])**
 

In addition to pmtwrf, function **simulate_pmt_response** also produces a vector **pmtblr([event] [npmt] [waveform])**. 

The so called basline restored (or BLR) waveforms correspond to the case of an ideal electronic which does not distort the input waveform (e.g, the photoelectron current). 

The output of **simulate_sipm_response** is a vector containing NSiPM waveforms (where NSiPM is the number of SiPMs in the tracking plane). Each waveform is a vector of (tipically) 1200 samples, corresponding to a DAQ window of 1.2 ms at a sampling rate of 1 MHz. This is stored in a 3D array called 
**sipmrwf([event] [npmt] [waveform])**



## simulate_pmt_response

1. Instances of **SPE** (single photoelectron) and **FEE** (front-end electronics) classes.

**spe = FE.SPE()**
       
**fee = FE.FEE(noise_FEEPMB_rms=FE.NOISE_I, noise_DAQ_rms=FE.NOISE_DAQ)**

2. Loop over all PMTs in the energy plane. For each PMT:

1. The input (true) current is computed as the convolution of the waveform of single photoelectrons (pmtrd) with the pulse (signal current) corresponding to a single photoelectron (SPE) 

**signal_i = FE.spe_pulse_from_vector(spe, pmtrd[event, pmt])**

2. The effect of the DAQ is simulated, decimating the input signal

**signal_d = FE.daq_decimator(FE.f_mc, FE.f_sample, signal_i)**

3. Effect of FEE and transform to adc counts

**signal_fee = FE.signal_v_fee(fee, signal_d, pmt) * FE.v_to_adc()**

4. Add noise daq
            
**signal_daq = cc * FE.noise_adc(fee, signal_fee)**

5. The RWF are baseline-shifted and sign-changed (to conform to data format)
**FE.OFFSET - signal_daq**

6. Compute BLR functions (LPF only no HPF)

Return a vector of RWF (one element per RWF) and a vector of BLR (one element per BLR)


## SiPM response

The simulation of the SiPM response simply adds noise (electronics and dark current noise) to the true photoelectrons and returns the total signal in adc counts

**dataSiPM = sipmrd[event] + sipms_noise_sampler.Sample()**

The noise sampler stores PDFs of each SiPM total noise. For each event those PDFs are sampled and the resulting noise added to the signal. 
        

In [ ]:
from __future__ import print_function
import sys
import os
from time import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math

In [ ]:
import invisible_cities.core.tbl_functions as tbl
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.core.wfm_functions as wfm
from invisible_cities.core.random_sampling\
     import NoiseSampler as SiPMsNoiseSampler
from invisible_cities.database import load_db
import invisible_cities.core.sensor_functions as snf

In [ ]:
from invisible_cities.cities.diomira import Diomira
import invisible_cities.sierpe.blr as blr
from   invisible_cities.core.system_of_units_c import SystemOfUnits

In [ ]:
DataPMT = load_db.DataPMT()

In [ ]:
units = SystemOfUnits()

In [ ]:
from invisible_cities.core.core_functions import define_window
from invisible_cities.core.core_functions import lrange

### Input  files

Required files (in IC_DATA)
1. **electrons_40keV_z250_MCRD.h5** 
2. **electrons_511keV_z250_MCRD.h5**
3. **electrons_1250keV_z250_MCRD.h5**
4. **electrons_2500keV_z250_MCRD.h5**


### Output  files

Will be writen in IC_DATA

1. **electrons_40keV_z250_RWF.h5** 
2. **electrons_511keV_z250_RWF.h5**
3. **electrons_1250keV_z250_RWF.h5**
4. **electrons_2500keV_z250_RWF.h5**



## Case 1: electrons of 40 keV

Electrons of 40 keV interacting in the chamber are a close approx to krypton decays. The S1 signal is quite wek, and the events are point-like, with narrow S2s. 

In [ ]:
MCRD_file = os.environ['IC_DATA']  + '/electrons_40keV_z250_MCRD.h5'
RWF_file = os.environ['IC_DATA']  + '/electrons_40keV_z250_RWF.h5'

### Running diomira

In [ ]:
def diomira_run(mcrd_file, rwf_file,
                sipm_noise_cut = 20 * units.pes,
                n_print        = 1,
                n_events       = 10):
    
    """Run Diomira."""

    # init machine
    fpp = Diomira()
    # set machine state 
    fpp.set_input_files([mcrd_file])
    fpp.set_output_file(rwf_file, compression='ZLIB4')
    fpp.set_print(nprint=n_print)
    fpp.set_sipm_noise_cut(noise_cut=sipm_noise_cut)

    # run for 10 evts
    t0 = time()
    nevt = fpp.run(nmax=n_events)
    t1 = time()
    dt = t1 - t0
    print("DIOMIRA run {} evts in {} s, time/event = {}".\
          format(nevt, dt, dt/nevt))

In [ ]:
diomira_run(MCRD_file, RWF_file)

#### Diomira: speed and file size:

runs at: ~0.5 s/event on i-7 processor 3.5 Ghz (speed almost certainly limited by disk access)
files size

1.1M ectrons_40keV_z250_MCRD.h5

1.5M electrons_40keV_z250_RWF.h5

MCRD files (produced by NEXUS MC): 110 kB/event
RWF files (output of DIOMIRA): 150 kB/event

##  Energy plane

### Raw waveforms

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(tb.open_file(RWF_file,'r+'))

In [ ]:
event = 0

In [ ]:
wfm.plot_pmt_waveforms(pmtrwf[event], zoom=False, window_size=400)

In [ ]:
wfm.plot_pmt_waveforms(pmtrwf[event], zoom=True, window_size=400)

In [ ]:
wfm.plot_waveforms_overlap(pmtrwf[event], zoom=True, window_size=400)

The set of plots above show, for one event:
1. The RWFs for each PMT, where the effect of the HPF is evident (the RWF are negative in sign and with a baseline at ~2500 adc counts, to emulate the behaviour of the DAQ in real data).
2. The RWF for the 12 PMTs of NEW superimposed, showing the differences (due to geometrical effects and calibration constants) between the response of each PMT. 

### Comparison between BLR and CWF

The input to the simulation of the energy plane are MCRD waveform, e.g, vectors of photoelectrons sampled each ns. NEW has 12 PMTs (NEXT-100 has around 60). The DAQ window is tipically ~1.2 ms. Thus, MCRD vectors contain 1.2 M samples. Most of those samples are zero, however, and can be handled efficiently by a smart compressor. IC relies on the PyTable library to write data in hdf5 format. Several compressors are available and can be set at the configuration level. ZLIB at compression level 4 yields good results. 

Diomira reads the MCRD vectors and produces raw waveforms (RWF) sampled each 40 ns. The RWF reflect the effect of the LPF and HPF filters introduced by the front-end electronics (FEE). Diomira also produce BLR (baseline restored) waveforms, which simply add the LPF filter. BLRs correspond to the response of an ideal FEE which does not distort the waveform. 

CWF (corrected waveforms) are the result of passing a blr (base line restorarion) algorithm to the RWFs. If the blr algorithm is perfect, then CWF and BLR waveforms must be identical.

The algorithm below does the following:
1) Reads the RWF and BLR waveforms in the RWF file produced by the previous cell
2) Computes CWF files passing a blr algorithm.
3) Returns the differences (in area) between CWF and BLR

In [ ]:
def compare_cwf_blr(n_evt=1, plot=False, window_size=500):
    """ 1) Reads the RWF and BLR waveforms in the RWF file produced by the previous cell
        2) Computes CWF files passing a blr algorithm.
        3) Returns the differences (in area) between CWF and BLR
    """
    
    coeff_c = DataPMT.coeff_c.values.astype(np.double)
    coeff_blr = DataPMT.coeff_blr.values.astype(np.double)
    
    if plot == True:
        plt.figure(figsize=(12, 12))
        
    DIFF = []
    for event in range(n_evt):
        CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr,
                             n_baseline=28000, thr_trigger=5)
        BLR = pmtblr[event]
        
        for i in range(len(BLR)):
            t0, t1 = define_window(BLR[i], window_size)
            if plot == True:
                plt.subplot(3, 4, i+1)
        
                mpl.set_plot_labels(xlabel="samples", ylabel="adc")
                plt.plot(BLR[i][t0:t1], label= 'BLR')
                plt.plot(CWF[i][t0:t1], label= 'CWF')   
                legend = plt.legend(loc='upper right')
                for label in legend.get_texts():
                    label.set_fontsize('small')
        
            diff = abs(np.sum(BLR[i][t0:t1]) - np.sum(CWF[i][t0:t1]))
            diff = 100. * diff/np.sum(BLR)
            DIFF.append(diff)
        
        plt.show()
            
    return np.array(DIFF)

In [ ]:
diff = compare_cwf_blr(n_evt=1, plot=True, window_size=120)

The set of plots above show, for one event:

1. The BLR functions, which only include the effect of the LPF (no distortion).
2. The CWF (in green) superimposed to the BLR (the eye cannot disitinguish one from the other)

In [ ]:
diff = compare_cwf_blr(n_evt=10, plot=False, window_size=120)

In [ ]:
mpl.histo(diff, nbins=10, 
          title="diff BLR-CWF", xlabel="abs(e[blr] - e[cwf])", ylabel="Frequency")

The histogram above shows the difference (in %) between the BLR function (e.g, only the effect of LPF) and the CWF (e.g, corrected waveform). The CWF is the result of producing a RWF (through DIOMIRA) and then passing a baseline restoration algorithm. There are 12 entries (one per PMT) for each of the 10 events in the example file. The tipical difference in absolute value between the CWF and the BLR is of the order of 0.04 % and always smaller than 0.1%. 

**draw a map of PMTS with signal for one event**

In [ ]:
snf.plot_sensor_list_ene_map(pmtblr[event], lrange(12), stype='PMT')

## Tracking plane

The tracking plane of NEW has 1792 SiPMs arranged in boards of 8x8 sensors. The SiPMs FEE runs at 1 MHz, providing one sample per microsecond.

The simulatin of the SiPms is very simple. Each MCRD contains the true number of pes in the sensor. Then, for each sensor, gaussian noise (corresponding to electronics and sensor thermal noise) and dark current noise is added. This is done through a SiPm noise sampler class, which stores the pdf of the noise in each sensor (measured from data) and returns a sample of this pdf each time that is invoked. 

In [ ]:
e40rd = tb.open_file(MCRD_file,'r+')
NEVENTS_DST, NSIPM, SIPMWL = e40rd.root.sipmrd.shape

In [ ]:
print('number of SiPM = {}, waveform length = {}'.format(NSIPM,SIPMWL))

In [ ]:
DataSiPM = load_db.DataSiPM(0)
sipm_adc_to_pes = DataSiPM.adc_to_pes.values.astype(np.double)

Conversion constants (number of adc counts per PES)

In [ ]:
plt.plot(sipm_adc_to_pes)

### Nb
A few constants have values equal to zero, corresponding to dead SiPMs in the data. The average number of adc counts per pes is very flat (e.g, about the same for all SiPMs) and near 16. 

In [ ]:
mpl.histo(sipm_adc_to_pes[sipm_adc_to_pes>0], nbins=20)

In [ ]:
np.mean(sipm_adc_to_pes[sipm_adc_to_pes>0])

In [ ]:
np.std(sipm_adc_to_pes[sipm_adc_to_pes>0])

Thus, one pes corresponds to 16 adc counts with a small rms of ~0.6 counts.

### Simulation of the SiPM response

It is instructive to reproduce the simulation of the SiPM response from the MCRD, to study the dependence with noise.

In [ ]:
def simulate_sipm_response(event, sipmrd, 
                           sipm_adc_to_pes,
                           sipms_noise_sampler):
        """Add noise with the NoiseSampler class and return
        the noisy waveform (in pes)."""
        # add noise (in PES) to true waveform
        dataSiPM = sipmrd[event] + sipms_noise_sampler.Sample()
        # return total signal in adc counts
        return wfm.to_adc(dataSiPM, sipm_adc_to_pes)

In [ ]:
noise_sampler = SiPMsNoiseSampler(SIPMWL, True)

In [ ]:
event=0
sipmrwf = simulate_sipm_response(event, 
                                 e40rd.root.sipmrd, 
                                 sipm_adc_to_pes,
                                 noise_sampler)

most of the SiPMs contain only noise

In [ ]:
plt.plot(sipmrwf[0])

###  NB: 
The noise distribution has an rms of about 3 counts, with dark current at the level of 1-2 pes (~30 adc counts) superimposed. 

#### Find the SiPMs with signal

select sipms with more than 20 pes (e.g, 300 adc counts)

In [ ]:
sipm_i = snf.sipm_with_signal(sipmrwf, thr=300)

In [ ]:
sipm_i

**draw a map of SiPms with signal**

In [ ]:
snf.plot_sensor_list_ene_map(sipmrwf, sipm_i, stype='SiPM')

plot SiPMs with signal 

In [ ]:
snf.plot_sipm_list(sipmrwf, sipm_i, x=4)

In [ ]:
esipm = np.array([np.sum(sipmrwf[i]) for i in sipm_i]) # in pes

In [ ]:
print(esipm/16.) # in adc counts

In [ ]:
mpl.histo(esipm/16., nbins=10)

#### NB
We can see  that there is 1 SiPM that takes most of the signal, while the others have almost 5 time less. 

#### Noise suppression

In Diomira SiPms are stored after noise suppression. This is done as follows.

First define sipm thresholds (in adc counts). As shown a cut on 20 pes is good 

In [ ]:
sipm_noise_cut = 20
sipms_thresholds = sipm_noise_cut *  sipm_adc_to_pes
plt.plot(sipms_thresholds)

Second compute the number of SiPMs above threshold

In [ ]:
sipmzs = wfm.noise_suppression(sipmrwf, sipms_thresholds)

The sipmzs vector replaces by exact zeros those samples below threshold (this allows the compressor in pytables to save storage space). Thus, for example, the first sipm has no samples above threshold.

In [ ]:
plt.plot(sipmzs[0])

while for sipm 696, we can see a clear signal above threshold. 

In [ ]:
plt.plot(sipmzs[696])

In [ ]:
nof_sipm = snf.sipm_with_signal(sipmzs)

In [ ]:
print('number of sipm with signal = {}'.format(nof_sipm))

In [ ]:
snf.plot_sipm_list(sipmzs, nof_sipm, x=4)

## Case 2: electrons of 511 keV

Electrons of 511 keV generated at a fixed point in the chamber have the energy of photoelectric interacions due to Na-22 decays. S1 is larger and S2 is no longer point like, but the track is still relatively short. 

### Input and output files

In [ ]:
MCRD_file = os.environ['IC_DATA']  + '/electrons_511keV_z250_MCRD.h5'
RWF_file = os.environ['IC_DATA']  + '/electrons_511keV_z250_RWF.h5'

### Running diomira

In [ ]:
diomira_run(MCRD_file, RWF_file)

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(tb.open_file(RWF_file,'r+'))

### Raw waveforms

In [ ]:
event = 0

In [ ]:
wfm.plot_pmt_waveforms(pmtrwf[event], zoom=False, window_size=400)

In [ ]:
wfm.plot_waveforms_overlap(pmtrwf[event], zoom=True, window_size=5000)

### Comparison between BLR and CWF

In [ ]:
diff = compare_cwf_blr(n_evt=1, plot=True, window_size=600)

In [ ]:
diff = compare_cwf_blr(n_evt=10, plot=False, window_size=600)

In [ ]:
mpl.histo(diff, nbins=10, 
          title="diff BLR-CWF", xlabel="abs(e[blr] - e[cwf])", ylabel="Frequency")

### SiPMs

In [ ]:
event=0
sipm_i = snf.sipm_with_signal(sipmrwf[event], thr=1) # already zs

In [ ]:
sipm_i

In [ ]:
snf.plot_sensor_list_ene_map(sipmrwf[event], sipm_i, stype='SiPM')

In [ ]:
snf.plot_sipm_list(sipmrwf[event], sipm_i, x=4)

## Case 3: electrons of 1250 keV

Electrons of intermediate energy, already a sizeable track

In [ ]:
MCRD_file = os.environ['IC_DATA']  + '/electrons_1250keV_z250_MCRD.h5'
RWF_file = os.environ['IC_DATA']  + '/electrons_125keV_z250_RWF.h5'

In [ ]:
diomira_run(MCRD_file, RWF_file)

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(tb.open_file(RWF_file,'r+'))

In [ ]:
event = 0

In [ ]:
wfm.plot_pmt_waveforms(pmtrwf[event], zoom=False, window_size=400)

In [ ]:
wfm.plot_waveforms_overlap(pmtrwf[event], zoom=True, window_size=5000)

In [ ]:
diff = compare_cwf_blr(n_evt=1, plot=True, window_size=2000)

In [ ]:
diff = compare_cwf_blr(n_evt=10, plot=False, window_size=2000)

In [ ]:
mpl.histo(diff, nbins=10, 
          title="diff BLR-CWF", xlabel="abs(e[blr] - e[cwf])", ylabel="Frequency")

In [ ]:
sipm_i = snf.sipm_with_signal(sipmrwf[event], thr=1) # already zs

In [ ]:
sipm_i

In [ ]:
snf.plot_sensor_list_ene_map(sipmrwf[event], sipm_i, stype='SiPM')

## Case 4: electrons of 2500 keV

Electrons of 2500 keV correspond to the energy of signal region

In [ ]:
MCRD_file = os.environ['IC_DATA']  + '/electrons_2500keV_z250_MCRD.h5'
RWF_file = os.environ['IC_DATA']  + '/electrons_250keV_z250_RWF.h5'

In [ ]:
diomira_run(MCRD_file, RWF_file)

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(tb.open_file(RWF_file,'r+'))
event = 0

In [ ]:
wfm.plot_pmt_waveforms(pmtrwf[event], zoom=False, window_size=400)

In [ ]:
wfm.plot_waveforms_overlap(pmtrwf[event], zoom=True, window_size=5000)

In [ ]:
diff = compare_cwf_blr(n_evt=1, plot=True, window_size=2000)

In [ ]:
diff = compare_cwf_blr(n_evt=10, plot=False, window_size=2000)

In [ ]:
mpl.histo(diff, nbins=10, 
          title="diff BLR-CWF", xlabel="abs(e[blr] - e[cwf])", ylabel="Frequency")

In [ ]:
sipm_i = snf.sipm_with_signal(sipmrwf[event], thr=1) # already zs

In [ ]:
sipm_i

In [ ]:
snf.plot_sensor_list_ene_map(sipmrwf[event], sipm_i, stype='SiPM')

In conclusion the distortion introduced by the FEE (which DIOMIRA simulates in great detail) can be corrected by the deconvolution algorithm to an accuracy of less that 1 per mil. The deconvolution algorithm works well at all energies.

A cut on some 20 pes in the SiPM energy supresses very effectively the noise leaving essentially only those SiPMs with signal and allowing very effective storage of the SiPM info. 

#### End of NB